In [283]:
# imports
import os
import pandas as pd

# constants
DATA_DIR = os.path.join('data','ad_placement')
train_path = os.path.join(DATA_DIR, 'census-income.csv')
test_path = os.path.join(DATA_DIR, 'census-income-test.csv')

with open(train_path, 'r') as train_file:
    df_train = pd.read_csv(train_file, header=None)
with open(test_path, 'r') as test_file:
    df_test = pd.read_csv(test_file, header=None)


In [284]:
list_var = [
    "age",
    "class_of_worker",
    "detailed_industry_recode",
    "detailed_occupation_recode",
    "education",
    "wage_per_hour",
    "enroll_in_edu_inst_last_wk",
    "marital_stat",
    "major_industry_code",
    "major_occupation_code",
    "race",
    "hispanic_origin",
    "sex",
    "member_of_a_labor_union",
    "reason_for_unemployment",
    "full_or_part_time_employment_stat",
    "capital_gains",
    "capital_losses",
    "dividends_from_stocks",
    "tax_filer_stat",
    "region_of_previous_residence",
    "state_of_previous_residence",
    "detailed_household_and_family_stat",
    "detailed_household_summary_in_household",
    "instance_weight",
    "migration_code_change_in_msa",
    "migration_code_change_in_reg",
    "migration_code_move_within_reg",
    "live_in_this_house_1_year_ago",
    "migration_prev_res_in_sunbelt",
    "num_persons_worked_for_employer",
    "family_members_under_18",
    "country_of_birth_father",
    "country_of_birth_mother",
    "country_of_birth_self",
    "citizenship",
    "own_business_or_self_employed",
    "fill_inc_questionnaire_for_veterans_admin",
    "veterans_benefits",
    "weeks_worked_in_year",
    "year",
    "income_bucket"
]

In [285]:
df_train.columns = list_var
df_test.columns = list_var

Y_train= df_train['income_bucket']
Y_test= df_test['income_bucket']

In [286]:
list_var_num = [
    "age",
    "wage_per_hour",
    "capital_gains",
    "capital_losses",
    "dividends_from_stocks",
    "num_persons_worked_for_employer",
    "weeks_worked_in_year"
]
list_var_cat =  [x for x in list_var if x not in list_var_num]

In [287]:
list_var_cat.remove("instance_weight")
list_var_cat.remove("income_bucket")
df_train=df_train.drop('instance_weight', 1)
df_test=df_test.drop('instance_weight', 1)
df_train=df_train.drop('income_bucket', 1)
df_test=df_test.drop('income_bucket', 1)

print df_train.shape
print df_test.shape
print len(list_var_cat)

(199523, 40)
(99762, 40)
33


In [288]:
import patsy
patsy_cat_formula = ""
for var in list_var_cat:
    if not patsy_cat_formula:
        patsy_cat_formula = var
    else:
        patsy_cat_formula = patsy_cat_formula + "+" + var

df_train_cat = patsy.dmatrix(patsy_cat_formula+"-1", data=df_train, return_type='dataframe')
df_test_cat = patsy.dmatrix(patsy_cat_formula+"-1", data=df_test, return_type='dataframe')

print df_train_cat.shape
print df_test_cat.shape

(199523, 374)
(99762, 373)


In [289]:
df_test_cat['detailed_household_and_family_stat[T. Grandchild <18 ever marr not in subfamily]']=0

In [290]:
print df_train_cat.shape
print df_test_cat.shape

(199523, 374)
(99762, 374)


In [291]:
df_train_model = pd.concat([df_train[list_var_num], df_train_cat], axis=1)
df_test_model = pd.concat([df_test[list_var_num], df_test_cat], axis=1)

In [292]:
print df_train_model.shape
print df_train.shape
print df_test_model.shape
print df_test.shape
#seems like df_test is missing a value in one of its categoric variables (376 = 375 + 1)

(199523, 381)
(199523, 40)
(99762, 381)
(99762, 40)


In [293]:
from sklearn import preprocessing

X_train = preprocessing.scale(df_train_model)
X_test = preprocessing.scale(df_test_model)
Y_train_bn = Y_train.replace(' - 50000.',0)
Y_train_bn = Y_train_bn.replace(' 50000+.',1)
Y_test_bn = Y_test.replace(' - 50000.',0)
Y_test_bn = Y_test_bn.replace(' 50000+.',1)

In [296]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression(penalty='l1')

In [297]:
X_new = logistic.fit_transform(X_train,Y_train_bn)

/Users/dkh320/anaconda2/lib/python2.7/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [299]:
X_new.shape
#logisticsticstic.fit()

(199523, 342)

In [312]:
np.logspace(-4, 4, 5)

array([  1.00000000e-04,   1.00000000e-02,   1.00000000e+00,
         1.00000000e+02,   1.00000000e+04])

In [309]:
logistic.coef_[0] == 0.0

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [234]:
Y_test_predict = logistic.predict(X_test)

In [235]:
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score
from sklearn import cross_validation

In [236]:
ac_scores = cross_validation.cross_val_score(logistic, X_train, Y_train, cv=5, scoring='accuracy')
pr_scores = cross_validation.cross_val_score(logistic, X_train, Y_train_bn, cv=5, scoring='precision')
rc_scores = cross_validation.cross_val_score(logistic, X_train, Y_train_bn, cv=5, scoring='recall')
f1_scores = cross_validation.cross_val_score(logistic, X_train, Y_train_bn, cv=5, scoring='f1')

In [237]:
print ac_scores
print pr_scores
print rc_scores
print f1_scores

[ 0.95246329  0.9520862   0.9519597   0.95185946  0.95271151]
[ 0.71260997  0.71714066  0.7154973   0.71199389  0.72159518]
[ 0.39241017  0.37666532  0.37479806  0.37641357  0.38731826]
[ 0.5061182   0.49391212  0.49191625  0.49247028  0.50407359]


In [247]:
print accuracy_score(Y_test_bn,Y_test_predict)
print precision_score(Y_test_bn,Y_test_predict)
print recall_score(Y_test_bn,Y_test_predict)
print f1_score(Y_test_bn,Y_test_predict)

0.0633207032738
0.0620803179797
0.999838344649
0.116902140528


In [258]:
import numpy as np
np.sort(logistic.coef_[0])[-1]

print len(logistic.coef_[0])

381


In [273]:
c_p={}
for i in range(len(df_test_model.columns)):
    c_p[str(df_train_model.columns[i])] = abs(logistic.coef_[0][i])

In [275]:
from operator import itemgetter
sorted_d=sorted(c_p.items(), key=itemgetter(1))

In [282]:
sorted_d[-10:]

[('citizenship[T. Native- Born in Puerto Rico or U S Outlying]',
  1.6293648136111485),
 ('citizenship[T. Native- Born in the United States]', 1.6560947421414585),
 ('age', 1.6749252556703524),
 ('fill_inc_questionnaire_for_veterans_admin[T. Not in universe]',
  1.8021506339316886),
 ('fill_inc_questionnaire_for_veterans_admin[T. Yes]', 1.9021515979729247),
 ('detailed_industry_recode', 1.9021515979729247),
 ('detailed_occupation_recode', 2.2184839823284324),
 ('own_business_or_self_employed', 2.7303296020555345),
 ('veterans_benefits', 3.0176606859479649),
 ('year', 3.022768678851365)]

In [280]:
sorted_d

[('detailed_household_summary_in_household[T. Other relative of householder]',
  2.48692624257026e-05),
 ('detailed_household_summary_in_household[T. Nonrelative of householder]',
  2.6760336518954841e-05),
 ('detailed_household_summary_in_household[T. Householder]',
  4.2529435657099309e-05),
 ('detailed_household_summary_in_household[T. Group Quarters- Secondary individual]',
  6.2540630851576633e-05),
 ('detailed_household_summary_in_household[T. Child under 18 never married]',
  7.4210442144908992e-05),
 ('detailed_household_summary_in_household[T. Child under 18 ever married]',
  8.9604147219361381e-05),
 ('detailed_household_summary_in_household[T. Spouse of householder]',
  9.9325492929337175e-05),
 ('migration_code_change_in_msa[T. Abroad to MSA]', 0.00018029629791836344),
 ('detailed_household_and_family_stat[T. Spouse of householder]',
  0.00022021899694060748),
 ('detailed_household_and_family_stat[T. Spouse of RP of unrelated subfamily]',
  0.00023859698721968763),
 ('detai

In [127]:
for  c in df_train_cat.columns:
    if c not in df_test_cat.columns:
        print c

detailed_household_and_family_stat[T. Grandchild <18 ever marr not in subfamily]


In [197]:
print df_train_model.columns
#print df_test_model.shape

Index([u'age', u'wage_per_hour', u'capital_gains', u'capital_losses',
       u'dividends_from_stocks', u'num_persons_worked_for_employer',
       u'weeks_worked_in_year', u'class_of_worker[ Federal government]',
       u'class_of_worker[ Local government]',
       u'class_of_worker[ Never worked]',
       ...
       u'citizenship[T. Native- Born abroad of American Parent(s)]',
       u'citizenship[T. Native- Born in Puerto Rico or U S Outlying]',
       u'citizenship[T. Native- Born in the United States]',
       u'fill_inc_questionnaire_for_veterans_admin[T. Not in universe]',
       u'fill_inc_questionnaire_for_veterans_admin[T. Yes]',
       u'detailed_industry_recode', u'detailed_occupation_recode',
       u'own_business_or_self_employed', u'veterans_benefits', u'year'],
      dtype='object', length=381)


In [59]:
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2

Y = df_train['income_bucket']
X = df_train_model

In [70]:
ss=SelectPercentile(chi2,percentile=5)

In [71]:
X_new=ss.fit_transform(X,Y)

In [100]:
used_columns = df_train_model.columns[ss.get_support()]

In [102]:
df_train_model[used_columns].head()

,age,wage_per_hour,capital_gains,capital_losses,dividends_from_stocks,num_persons_worked_for_employer,weeks_worked_in_year,education[T. Bachelors degree(BA AB BS)],education[T. Masters degree(MA MS MEng MEd MSW MBA)],education[T. Prof school degree (MD DDS DVM LLB JD)],major_industry_code[T. Not in universe or children],major_occupation_code[T. Executive admin and managerial],major_occupation_code[T. Professional specialty],tax_filer_stat[T. Joint both under 65],tax_filer_stat[T. Nonfiler],detailed_household_and_family_stat[T. Householder],detailed_household_summary_in_household[T. Householder],income_bucket[T. 50000+.],detailed_industry_recode
0,73,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,58,0,0,0,0,1,52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0
2,18,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,9,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,10,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [104]:
X_new[0] == df_train_model[used_columns].iloc[0]

age                                                         True
wage_per_hour                                               True
capital_gains                                               True
capital_losses                                              True
dividends_from_stocks                                       True
num_persons_worked_for_employer                             True
weeks_worked_in_year                                        True
education[T. Bachelors degree(BA AB BS)]                    True
education[T. Masters degree(MA MS MEng MEd MSW MBA)]        True
education[T. Prof school degree (MD DDS DVM LLB JD)]        True
major_industry_code[T. Not in universe or children]         True
major_occupation_code[T. Executive admin and managerial]    True
major_occupation_code[T. Professional specialty]            True
tax_filer_stat[T. Joint both under 65]                      True
tax_filer_stat[T. Nonfiler]                                 True
detailed_household_and_fa

In [198]:
X_new.shape

(199523, 19)